In [ ]:
# Download audio  transcripts from 10 youtube videos.
#Use the method created above to generate a ‘vector store’ based on the transcripts of these videos.
#Create a vector store and embeddings based on this data.
#Document loading
#Document splitting
#And Vector store section

def parse_transcript_file(file_path, interval_seconds):
  result = []
  with open(file_path, 'I') as file:
    lines = file.readlines()
  start_time = 0
  end_time = interval_seconds
  current_text = ""

  for line in lines:
        time_str, text = line.split(',', 1)
        # The line below calculates the total time in seconds from a time string in the format "mm:ss"
        # time_str is a string representing the time in the format "mm:ss"
        # The strip() function removes any leading/trailing whitespace
        # The split(':') function splits the string into a list of two elements, representing minutes and seconds
        # The zip function pairs the elements of the list "[60, 1]" with the elements of the time string's list obtained in the previous step

        time_in_seconds = sum(x * int(t) for x, t in zip([60, 1], time_str.strip().split(':')))
        if time_in_seconds < end_time:
            current_text += text.strip()
        else:
            result.append((f"{start_time // 60:02}:{start_time % 60:02}",
                           f"{end_time // 60:02}:{end_time % 60:02}", current_text))
            current_text = text.strip()
            start_time = end_time
            end_time += interval_seconds

  if current_text:
        result.append((f"{start_time // 60:02}:{start_time % 60:02}",
                       f"{end_time // 60:02}:{end_time % 60:02}", current_text))
        return result;

file_path = "video1.txt"
interval_seconds = 30
result = parse_transcript_file(file_path, interval_seconds)
print(result)



import tensorflow as tf
import tensorflow_hub as hub

def load_transcript(file_path):
    with open(file_path, 'r') as file:
        return file.readlines()

def create_vector_store(transcripts, interval_seconds):
    vector_store = {}
    embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

    for transcript in transcripts:
        parsed_intervals = parse_transcript_file(transcript, interval_seconds)
        for start_time, end_time, text in parsed_intervals:
            # Concatenate start time and end time to create a unique key for each interval
            key = f"{start_time}_{end_time}"
            embedding = embed([text])[0].numpy()
            vector_store[key] = embedding

    return vector_store

#usage:
transcript_files = ["video1.txt"]
interval_seconds = 30

transcripts = [load_transcript(file) for file in transcript_files]
vector_store = create_vector_store(transcripts, interval_seconds)








